In [1]:


import plotly.graph_objects as go

In [2]:
df = yf.download('SBIN.NS', interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [3]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930
2024-08-07,809.000000,810.700012,800.549988,808.650024,808.650024,17692355
2024-08-08,812.000000,815.900024,802.799988,812.000000,812.000000,6793925


In [4]:
def calculate_sma(data, length:int):
    return ta.sma(data['Close'],length=length)

df['SMA'] = calculate_sma(df, 20)
# df.dropna(inplace=True)

In [5]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA
Date,,,,,,,
1996-01-01,18.691147,18.978922,18.540184,18.823240,12.205255,43733533,NaN
1996-01-02,18.894005,18.964767,17.738192,18.224106,11.816770,56167280,NaN
1996-01-03,18.327892,18.568489,17.643839,17.738192,11.501696,68296318,NaN
1996-01-04,17.502312,17.832542,17.223972,17.676863,11.461928,86073880,NaN
1996-01-05,17.738192,17.785366,17.459852,17.577793,11.397692,76613039,NaN
...,...,...,...,...,...,...,...
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501


In [6]:
def calculate_slope(series,period:int = 5):
    slopes = [0 for _ in range(period-1)]
    for i in range(period-1,len(series)):
        x = np.arange(period)
        y = series[i-period+1:i+1].values
        slope = np.polyfit(x,y,1)[0]
        percent_slope = (slope/y[0]) *100
        slopes.append(percent_slope)
    return slopes

In [7]:
df['Slope'] = calculate_slope(df['SMA'])

In [8]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope
Date,,,,,,,,
1996-01-01,18.691147,18.978922,18.540184,18.823240,12.205255,43733533,NaN,0.000000
1996-01-02,18.894005,18.964767,17.738192,18.224106,11.816770,56167280,NaN,0.000000
1996-01-03,18.327892,18.568489,17.643839,17.738192,11.501696,68296318,NaN,0.000000
1996-01-04,17.502312,17.832542,17.223972,17.676863,11.461928,86073880,NaN,0.000000
1996-01-05,17.738192,17.785366,17.459852,17.577793,11.397692,76613039,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999,0.165977
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000,0.052230
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501,-0.110109


In [9]:
df['SMA_10'] = calculate_sma(df,10)
df['SMA_20'] = calculate_sma(df,20)
df['SMA_50'] = calculate_sma(df,50)

In [10]:
dfpl = df[:]
fig = go.Figure(data= [go.Candlestick(
    x=dfpl.index,
    open=dfpl['Open'],
    high=dfpl['High'],
    low=dfpl['Low'],
    close=dfpl['Close'],
)])

fig.add_scatter(x=dfpl.index, y=dfpl['SMA_10'], mode='markers', name='SMA 10', marker=dict(color='blue', size=2, ))
fig.add_scatter(x=dfpl.index, y=dfpl['SMA_20'], mode='markers', name='SMA 20', marker=dict(color='red', size=2, ))
fig.add_scatter(x=dfpl.index, y=dfpl['SMA_50'], mode='markers', name='SMA 50', marker=dict(color='green', size=2, ))

fig.show()

In [11]:
def determine_trend(data):
    if data['SMA_10'] > data['SMA_20']  > data['SMA_50'] and data['Close'] > data['VWAP']:
        return 2
    elif data['SMA_10'] < data['SMA_20']  < data['SMA_50'] and data['Close'] < data['VWAP']:
        return 1
    else:
        return 0

In [12]:
def check_candles(data,backcandles,ma_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles,len(data )):
        if all(data['Close'][i-backcandles:i] > data[ma_column][i-backcandles:i]):
            categories.append(2)
        elif all(data['Close'][i-backcandles:i] < data[ma_column][i-backcandles:i]):
            categories.append(1)
        else:
            categories.append(0)
    return categories

df['Category'] = check_candles(df,10,'SMA_20')
    

In [13]:
df.tail(10)

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_50,Category
Date,,,,,,,,,,,,
2024-07-26,844.650024,864.250000,844.000000,862.450012,862.450012,12081506,859.380002,0.080584,870.829999,859.380002,842.808000,0
2024-07-29,866.049988,889.099976,863.200012,871.599976,871.599976,26582781,860.512500,0.075541,872.019995,860.512500,843.834000,0
2024-07-30,874.700012,880.549988,869.000000,872.799988,872.799988,13528447,862.054999,0.111441,871.164996,862.054999,845.051000,0
2024-07-31,873.849976,874.950012,868.000000,872.400024,872.400024,11636874,864.367499,0.168849,870.334998,864.367499,846.142001,0
2024-08-01,877.799988,881.400024,859.450012,862.650024,862.650024,11787203,865.502499,0.187344,867.245001,865.502499,846.782001,0
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999,0.165977,863.095001,865.929999,847.364000,0
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000,0.052230,856.580005,863.525000,846.955001,0
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501,-0.110109,849.960004,860.597501,846.337002,0
2024-08-07,809.000000,810.700012,800.549988,808.650024,808.650024,17692355,857.965002,-0.235788,845.625006,857.965002,845.836002,0


In [14]:
df['VWAP'] = ta.vwap(df['High'], df['Low'], df['Close'], df['Volume'])


In [15]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_50,Category,VWAP
Date,,,,,,,,,,,,,
1996-01-01,18.691147,18.978922,18.540184,18.823240,12.205255,43733533,NaN,0.000000,NaN,NaN,NaN,0,18.780782
1996-01-02,18.894005,18.964767,17.738192,18.224106,11.816770,56167280,NaN,0.000000,NaN,NaN,NaN,0,18.309022
1996-01-03,18.327892,18.568489,17.643839,17.738192,11.501696,68296318,NaN,0.000000,NaN,NaN,NaN,0,17.983507
1996-01-04,17.502312,17.832542,17.223972,17.676863,11.461928,86073880,NaN,0.000000,NaN,NaN,NaN,0,17.577792
1996-01-05,17.738192,17.785366,17.459852,17.577793,11.397692,76613039,NaN,NaN,NaN,NaN,NaN,0,17.607670
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999,0.165977,863.095001,865.929999,847.364000,0,850.766663
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000,0.052230,856.580005,863.525000,846.955001,0,814.333333
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501,-0.110109,849.960004,860.597501,846.337002,0,806.633341


In [16]:
df.dropna()

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_50,Category,VWAP
Date,,,,,,,,,,,,,
1996-03-08,22.649218,22.734135,22.276527,22.635065,14.676898,39637704,22.999028,0.041767,23.102579,22.999028,20.537146,0,22.548576
1996-03-11,22.172739,22.361443,21.837790,22.116129,14.340410,39109363,22.968599,-0.058340,22.929443,22.968599,20.603004,0,22.105120
1996-03-12,22.092541,22.186892,21.889683,22.083105,14.318999,27429706,22.930386,-0.157532,22.757250,22.930386,20.680184,0,22.053226
1996-03-13,22.073669,22.686958,21.960447,22.611477,14.661597,27421227,22.885098,-0.193719,22.635065,22.885098,20.777649,0,22.419628
1996-03-14,22.691675,22.889814,22.465231,22.587889,14.646307,36343129,22.845470,-0.169841,22.626573,22.845470,20.875870,1,22.647645
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999,0.165977,863.095001,865.929999,847.364000,0,850.766663
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000,0.052230,856.580005,863.525000,846.955001,0,814.333333
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501,-0.110109,849.960004,860.597501,846.337002,0,806.633341


In [17]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_50,Category,VWAP
Date,,,,,,,,,,,,,
1996-01-01,18.691147,18.978922,18.540184,18.823240,12.205255,43733533,NaN,0.000000,NaN,NaN,NaN,0,18.780782
1996-01-02,18.894005,18.964767,17.738192,18.224106,11.816770,56167280,NaN,0.000000,NaN,NaN,NaN,0,18.309022
1996-01-03,18.327892,18.568489,17.643839,17.738192,11.501696,68296318,NaN,0.000000,NaN,NaN,NaN,0,17.983507
1996-01-04,17.502312,17.832542,17.223972,17.676863,11.461928,86073880,NaN,0.000000,NaN,NaN,NaN,0,17.577792
1996-01-05,17.738192,17.785366,17.459852,17.577793,11.397692,76613039,NaN,NaN,NaN,NaN,NaN,0,17.607670
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999,0.165977,863.095001,865.929999,847.364000,0,850.766663
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000,0.052230,856.580005,863.525000,846.955001,0,814.333333
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501,-0.110109,849.960004,860.597501,846.337002,0,806.633341


In [18]:
df.ta.bbands(
    length=20,std=2,append=True)

,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
Date,,,,,
1996-01-01,NaN,NaN,NaN,NaN,NaN
1996-01-02,NaN,NaN,NaN,NaN,NaN
1996-01-03,NaN,NaN,NaN,NaN,NaN
1996-01-04,NaN,NaN,NaN,NaN,NaN
1996-01-05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2024-08-02,839.767661,865.929999,892.092337,6.042599,0.154465
2024-08-05,828.269412,863.525000,898.780588,8.165505,-0.235699
2024-08-06,815.158793,860.597501,906.036208,10.559805,-0.192114


In [19]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_50,Category,VWAP,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
Date,,,,,,,,,,,,,,,,,,
1996-01-01,18.691147,18.978922,18.540184,18.823240,12.205255,43733533,NaN,0.000000,NaN,NaN,NaN,0,18.780782,NaN,NaN,NaN,NaN,NaN
1996-01-02,18.894005,18.964767,17.738192,18.224106,11.816770,56167280,NaN,0.000000,NaN,NaN,NaN,0,18.309022,NaN,NaN,NaN,NaN,NaN
1996-01-03,18.327892,18.568489,17.643839,17.738192,11.501696,68296318,NaN,0.000000,NaN,NaN,NaN,0,17.983507,NaN,NaN,NaN,NaN,NaN
1996-01-04,17.502312,17.832542,17.223972,17.676863,11.461928,86073880,NaN,0.000000,NaN,NaN,NaN,0,17.577792,NaN,NaN,NaN,NaN,NaN
1996-01-05,17.738192,17.785366,17.459852,17.577793,11.397692,76613039,NaN,NaN,NaN,NaN,NaN,0,17.607670,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999,0.165977,863.095001,865.929999,847.364000,0,850.766663,839.767661,865.929999,892.092337,6.042599,0.154465
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000,0.052230,856.580005,863.525000,846.955001,0,814.333333,828.269412,863.525000,898.780588,8.165505,-0.235699
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501,-0.110109,849.960004,860.597501,846.337002,0,806.633341,815.158793,860.597501,906.036208,10.559805,-0.192114


In [20]:
df.dropna(inplace=True)

In [21]:
df['Trend'] = df.apply(determine_trend
                        ,axis=1)

In [22]:
df['entry'] = 0
buy_entry_condition = (df['Trend'] == 2) & ((df['Open'] < df['BBL_20_2.0']) & (df['Close'] > df['BBL_20_2.0']))
df.loc[buy_entry_condition,'entry'] = 2

In [23]:
sell_entry_condition = (df['Trend'] == 1) & ((df['Open'] > df['BBU_20_2.0']) & (df['Close'] < df['BBU_20_2.0']))
df.loc[sell_entry_condition,'entry'] = 1

In [24]:
df[df['entry'] ==2]

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,SMA_50,Category,VWAP,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,Trend,entry
Date,,,,,,,,,,,,,,,,,,,,
1997-07-17,31.183363,32.070274,31.070141,31.782499,21.133089,210860206,32.548874,0.114403,32.700073,32.548874,30.703206,0,31.640972,31.377526,32.548874,33.720222,7.197472,0.172866,2,2
2003-01-09,25.286358,27.079048,25.286358,26.965824,19.965607,24085922,26.607523,0.119032,26.751174,26.607523,24.844602,0,26.443743,25.909966,26.607523,27.305081,5.243310,0.756825,2,2
2005-06-17,54.158096,63.782009,54.158096,62.970581,49.830036,20822232,63.620902,0.316602,64.467476,63.620902,60.707357,2,60.303562,61.347257,63.620902,65.894546,7.147477,0.356987,2,2
2007-10-17,161.342072,176.759201,156.718811,172.546371,140.555878,19711011,176.107458,0.658695,178.079181,176.107458,160.525824,2,168.674795,164.978980,176.107458,187.235937,12.638282,0.340001,2,2
2009-01-13,114.500000,118.629997,114.040001,117.489998,97.082886,22844560,125.870749,0.091882,126.627500,125.870749,119.609300,0,116.719999,115.620074,125.870749,136.121425,16.287621,0.091210,2,2
2014-12-17,294.399994,306.399994,287.000000,301.750000,274.362305,28290440,310.610001,0.226086,310.654996,310.610001,281.932000,0,298.383331,295.667461,310.610001,325.552541,9.621416,0.203531,2,2
2020-01-08,312.100006,321.500000,311.000000,319.799988,301.009949,44527485,329.432500,0.015036,329.989999,329.432500,325.548998,0,317.433329,315.135948,329.432500,343.729052,8.679503,0.163118,2,2
2024-03-18,727.099976,737.900024,722.099976,730.950012,718.894409,18145126,759.840002,-0.050168,761.250006,759.840002,697.331003,0,730.316671,727.517830,759.840002,792.162175,8.507626,0.053093,2,2


In [25]:
df['RSI'] = ta.rsi(df['Close'], length=14)

In [26]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,...,Category,VWAP,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,Trend,entry,RSI
Date,,,,,,,,,,,,,,,,,,,,,
1996-03-08,22.649218,22.734135,22.276527,22.635065,14.676898,39637704,22.999028,0.041767,23.102579,22.999028,...,0,22.548576,22.088184,22.999028,23.909871,7.920714,0.300206,2,0,NaN
1996-03-11,22.172739,22.361443,21.837790,22.116129,14.340410,39109363,22.968599,-0.058340,22.929443,22.968599,...,0,22.105120,21.985348,22.968599,23.951850,8.561701,0.066504,0,0,NaN
1996-03-12,22.092541,22.186892,21.889683,22.083105,14.318999,27429706,22.930386,-0.157532,22.757250,22.930386,...,0,22.053226,21.874535,22.930386,23.986238,9.209191,0.098769,0,0,NaN
1996-03-13,22.073669,22.686958,21.960447,22.611477,14.661597,27421227,22.885098,-0.193719,22.635065,22.885098,...,0,22.419628,21.856469,22.885098,23.913726,8.989501,0.366997,0,0,NaN
1996-03-14,22.691675,22.889814,22.465231,22.587889,14.646307,36343129,22.845470,-0.169841,22.626573,22.845470,...,1,22.647645,21.835328,22.845470,23.855612,8.843259,0.372503,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999,0.165977,863.095001,865.929999,...,0,850.766663,839.767661,865.929999,892.092337,6.042599,0.154465,0,0,45.565691
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000,0.052230,856.580005,863.525000,...,0,814.333333,828.269412,863.525000,898.780588,8.165505,-0.235699,0,0,34.911583
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501,-0.110109,849.960004,860.597501,...,0,806.633341,815.158793,860.597501,906.036208,10.559805,-0.192114,0,0,31.823569


In [27]:
def rsi_signal(data):
    if (data['RSI'] < 45) and (data['Close'] < data['BBL_20_2.0']) and data['Trend'] == 1:
        return 1
    elif (data['RSI'] > 55) and (data['Close'] > data['BBU_20_2.0']) and data['Trend'] == 2:
        return 2
    else:
        return 0
    
df['RSI Signal'] = df.apply(rsi_signal, axis=1)

In [28]:
def rejection_candles(data):
    def identify_rejection_candle(row):
        body_size = abs(row['Close'] - row['Open'])
        lower_wick_size = min(row['Open'], row['Close']) - row['Low']
        upper_wick_size = row['High'] - max(row['Open'], row['Close'])

        # Criteria for rejection candle
        is_rejection_candle = (
            (lower_wick_size > 1.5 * body_size) and
            (upper_wick_size < 0.8 * body_size) and
            (body_size > 0.01 * row['Open'])
        )

        if is_rejection_candle:
            # Criteria for strong rejection candle
            return 2 if lower_wick_size > 2 * body_size else 1
        return 0

    data['Rejection Signal'] = data.apply(identify_rejection_candle, axis=1)
    return data


In [30]:
df = rejection_candles(df)

In [31]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,...,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,Trend,entry,RSI,RSI Signal,Rejection Signal
Date,,,,,,,,,,,,,,,,,,,,,
1996-03-08,22.649218,22.734135,22.276527,22.635065,14.676898,39637704,22.999028,0.041767,23.102579,22.999028,...,22.088184,22.999028,23.909871,7.920714,0.300206,2,0,NaN,0,0
1996-03-11,22.172739,22.361443,21.837790,22.116129,14.340410,39109363,22.968599,-0.058340,22.929443,22.968599,...,21.985348,22.968599,23.951850,8.561701,0.066504,0,0,NaN,0,0
1996-03-12,22.092541,22.186892,21.889683,22.083105,14.318999,27429706,22.930386,-0.157532,22.757250,22.930386,...,21.874535,22.930386,23.986238,9.209191,0.098769,0,0,NaN,0,0
1996-03-13,22.073669,22.686958,21.960447,22.611477,14.661597,27421227,22.885098,-0.193719,22.635065,22.885098,...,21.856469,22.885098,23.913726,8.989501,0.366997,0,0,NaN,0,0
1996-03-14,22.691675,22.889814,22.465231,22.587889,14.646307,36343129,22.845470,-0.169841,22.626573,22.845470,...,21.835328,22.845470,23.855612,8.843259,0.372503,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-02,856.700012,858.299988,846.150024,847.849976,847.849976,12609331,865.929999,0.165977,863.095001,865.929999,...,839.767661,865.929999,892.092337,6.042599,0.154465,0,0,45.565691,0,0
2024-08-05,830.000000,831.349976,800.000000,811.650024,811.650024,27676951,863.525000,0.052230,856.580005,863.525000,...,828.269412,863.525000,898.780588,8.165505,-0.235699,0,0,34.911583,0,0
2024-08-06,822.650024,826.549988,795.650024,797.700012,797.700012,17900930,860.597501,-0.110109,849.960004,860.597501,...,815.158793,860.597501,906.036208,10.559805,-0.192114,0,0,31.823569,0,0


In [36]:
#Find where all Signals are equal to 2

dfpl = df[df['Trend'] == 2]
dfpl = dfpl[dfpl['Rejection Signal'] == 2]

In [37]:
dfpl

,Open,High,Low,Close,Adj Close,Volume,SMA,Slope,SMA_10,SMA_20,...,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,Trend,entry,RSI,RSI Signal,Rejection Signal
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-29,21.984035,21.984035,20.757458,21.606627,15.243669,26620577,20.738587,0.982442,21.220255,20.738587,...,19.363702,20.738587,22.113473,13.259198,0.815677,2,0,66.403872,0,2
2004-04-27,61.309982,61.309982,51.044476,59.446529,44.974487,44040288,59.293916,0.577937,60.369293,59.293916,...,56.444033,59.293916,62.143798,9.612731,0.526775,2,0,53.419969,0,2
2004-12-09,55.195969,57.724606,47.553452,57.144341,44.388298,53101625,50.514926,0.900179,52.960767,50.514926,...,44.061061,50.514926,56.968790,25.552306,1.013600,2,0,80.097245,2,2
2004-12-14,60.857094,60.857094,47.836506,56.946201,44.234386,34413633,51.958748,0.990083,55.105391,51.958748,...,45.162653,51.958748,58.754844,26.159583,0.866935,2,0,75.576106,0,2
2007-10-03,182.382584,183.892212,168.512817,179.981323,146.612381,16419202,164.441530,0.925469,174.135736,164.441530,...,143.246734,164.441530,185.636325,25.777911,0.866595,2,0,70.071667,0,2
2010-11-18,310.799988,311.100006,296.500000,306.505005,261.117157,43440440,319.865251,-0.130586,321.302502,319.865251,...,300.059324,319.865251,339.671179,12.383919,0.162721,2,0,42.785999,0,2
2020-12-22,254.800003,258.700012,248.250000,257.450012,242.323380,51189571,260.537502,0.389370,267.790004,260.537502,...,237.917333,260.537502,283.157670,17.364232,0.431754,2,0,53.558142,0,2
